In [1]:
import open3d as o3d
import numpy as np
import copy
from utils.camera_utils import RealCamera, RealCameraROS
import cv2
from cv_bridge import CvBridge, CvBridgeError
from contact_grasp.srv import segmentationSrv, segmentationSrvResponse
from utils.visualisation_utils import depth2pc
import rospy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2023-06-02 17:09:20,492 - topics - topicmanager initialized


In [83]:
# TODO get image and depth from ros


camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()

    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    intrinsic, distortion = camera.getIntrinsic()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")

rgb, depth_image, depth_scale = camera.get_rgb_depth()
depth = depth_image * depth_scale

# TODO segment ROS image

#convert to ros image
bridge = CvBridge()
ros_img = bridge.cv2_to_imgmsg(rgb, encoding="rgb8")
try:
    segmentation = rospy.ServiceProxy("segmentation", segmentationSrv)
    resp = segmentation(ros_img)
except rospy.ServiceException as e:
    print(e)
#convert to cv image
segmap = bridge.imgmsg_to_cv2(resp.image, desired_encoding="passthrough")

#show on cv2
# cv2.imshow("rgb", rgb)
# cv2.imshow("depth", depth_image)
# cv2.imshow("segmap", segmap)
# cv2.waitKey(5000)
# cv2.destroyAllWindows()

kernel = np.ones((24, 24),np.uint8)
erosion = cv2.erode(segmap,kernel,iterations = 1)
#TODO get point cloud from depth image


pc, pc_colors = depth2pc(depth, intrinsic, rgb, segmap=erosion)
mean_pos_pc = np.mean(pc, axis=0)
pc_colors[:,[0,2]]=pc_colors[:,[2,0]]
#TODO get init translation and rotation

Camera topic found


In [84]:
#visualize pc
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc)
pcd.colors = o3d.utility.Vector3dVector(pc_colors/255)
o3d.visualization.draw_geometries([pcd])


In [85]:
def draw_registration_result(source, target, transformation, mean_pos_pc, source_color=None):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)

    target_temp.paint_uniform_color([0, 0.651, 0.929])
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
    mesh_frame_2 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05, origin=mean_pos_pc)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp, mesh_frame, mesh_frame_2])

if __name__ == "__main__":
    
    #axis
    source_mesh = o3d.io.read_triangle_mesh("../todel/apple.stl")
    target_mesh = o3d.io.read_triangle_mesh("../todel/apple.stl")
    source = source_mesh.sample_points_uniformly(number_of_points=5000)
    # truncate source
    # source = source.crop(source.get_axis_aligned_bounding_box())
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(pc)
    source.colors = o3d.utility.Vector3dVector(pc_colors/255)
    target = target_mesh.sample_points_uniformly(number_of_points=5000)
    target_size = np.linalg.norm(target.get_max_bound() - target.get_min_bound())
    source_size = np.linalg.norm(source.get_max_bound() - source.get_min_bound())
    target.scale((1.3*source_size) / target_size, center=target.get_center())
    threshold = 0.02
    # trans_init = np.asarray([[0.862, 0.011, -0.507, -mean_pos_pc[0]],
    #                          [-0.139, 0.967, -0.215, -mean_pos_pc[1]],
    #                          [0.487, 0.255, 0.835,  -mean_pos_pc[2]], 
    #                          [0.0, 0.0, 0.0, 1.0]])
    trans_init = np.asarray([[1.0, 0.0, 0.0, -mean_pos_pc[0]],
                                [0.0, 1.0, 0.0, -mean_pos_pc[1]],
                                [0.0, 0.0, 1.0,  -mean_pos_pc[2]],
                                [0.0, 0.0, 0.0, 1.0]])
    draw_registration_result(source, target, trans_init, mean_pos_pc, pc_colors/255)
    print("Initial alignment")
    evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, trans_init)
    print(evaluation)

    # print("Apply point-to-point ICP")
    # reg_p2p = o3d.pipelines.registration.registration_icp(
    #     source, target, threshold, trans_init,
    #     o3d.pipelines.registration.TransformationEstimationPointToPoint())
    # print(reg_p2p)
    # print("Transformation is:")
    # print(reg_p2p.transformation)
    # print("")
    # draw_registration_result(source, target, reg_p2p.transformation, mean_pos_pc)

    print("Apply point-to-plane ICP")
    reg_p2l = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    print(reg_p2l)
    print("Transformation is:")
    print(reg_p2l.transformation)
    print("")
    draw_registration_result(source, target, reg_p2l.transformation,mean_pos_pc, pc_colors/255)

Initial alignment
RegistrationResult with fitness=1.000000e+00, inlier_rmse=1.033591e-02, and correspondence_set size of 3571
Access transformation to get result.
Apply point-to-plane ICP
RegistrationResult with fitness=1.000000e+00, inlier_rmse=1.728863e-03, and correspondence_set size of 3571
Access transformation to get result.
Transformation is:
[[ 0.97445937  0.05183872 -0.2184987   0.09271508]
 [ 0.05124813  0.89598021  0.44112707 -0.3929574 ]
 [ 0.21863798 -0.44105806  0.87043967 -0.58889541]
 [ 0.          0.          0.          1.        ]]

